In [31]:
import pandas as pd
import numpy as np
import sklearn.linear_model as sklm

In [85]:
headers = pd.read_csv("../data/2018_headers.csv").columns.values |> list
["key", "level", "event"] |> headers.extend

In [86]:
KEY='2018vapor'

In [87]:
scores = pd.read_csv(f"../data/matches_{KEY}.csv", names=headers)
scores.index = scores.key
teams = pd.read_csv(f"../data/matches_teams_{KEY}.csv", names=['team1', 'team2', 'team3', 'key', 'level', 'event'])
teams.index = teams.key

scores_train = scores[scores.level == "qm"].select_dtypes(include='number')
teams_train = teams[teams.level == "qm"]

In [88]:
def make_beta(teams: pd.DataFrame):
    teamnums = teams.team1.values |> np.unique
    beta = pd.DataFrame(0, index = teams.key, columns = teamnums)
    for (key, row) in teams.iterrows():
        for col in ['team1', 'team2', 'team3']:
            t = row[col]
            beta.at[key,t] = 1
    return (beta, teamnums)

(beta, teamnums) = make_beta(teams_train)

In [89]:
components = pd.DataFrame(index=teamnums)
rsquared = pd.Series(index=scores_train.columns)

In [90]:
for (column, y) in scores_train.iteritems():
    model = sklm.LinearRegression(fit_intercept=False).fit(beta, y)
    components[column] = model.coef_
    rsquared[column] = model.score(beta, y)

In [91]:
rsquared

adjustPoints                1.000000
autoOwnershipPoints         0.457037
autoPoints                  0.430432
autoRunPoints               0.237905
autoScaleOwnershipSec       0.252798
autoSwitchOwnershipSec      0.459405
endgamePoints               0.411276
foulCount                   0.200737
foulPoints                  0.262136
rp                          0.488638
techFoulCount               0.243883
teleopOwnershipPoints       0.463999
teleopPoints                0.540281
teleopScaleBoostSec         0.462627
teleopScaleForceSec         0.362356
teleopScaleOwnershipSec     0.446493
teleopSwitchBoostSec        0.414402
teleopSwitchForceSec        0.253344
teleopSwitchOwnershipSec    0.343056
totalPoints                 0.532936
vaultBoostPlayed            0.516430
vaultBoostTotal             0.523577
vaultForcePlayed            0.379213
vaultForceTotal             0.383610
vaultLevitatePlayed         0.370745
vaultLevitateTotal          0.340440
vaultPoints                 0.524560
d

In [92]:
components.sum(1).sort_values()

frc5954     30.537732
frc3359     31.795357
frc7330     44.433959
frc617      49.190501
frc539      92.706527
frc435     113.849702
frc4286    114.154797
frc6021    126.658575
frc2998    196.338339
frc6189    200.666394
frc1908    209.018290
frc1080    220.168723
frc2068    244.984428
frc6312    260.506990
frc3389    276.898603
frc5957    300.141363
frc1446    306.650207
frc1793    307.354422
frc1137    311.595594
frc6194    344.141773
frc6882    362.823039
frc5950    371.235099
frc5724    373.371617
frc2890    382.395809
frc422     402.520987
frc977     422.284753
frc3072    423.474316
frc2028    424.249913
frc1522    441.058756
frc4638    449.961101
frc5804    479.669781
frc3136    497.836282
frc122     528.923174
frc6334    556.307100
frc5546    561.309339
frc2363    589.210261
frc1610    658.076395
dtype: float64

In [93]:
model = sklm.BayesianRidge(fit_intercept=False).fit(beta, scores_train.totalPoints)
comp = pandas.DataFrame(components.totalPoints)
comp["bayes"] = model.coef_
comp.bayes - comp.totalPoints |> .sort_values()

frc6334   -6.982591
frc2363   -6.052494
frc1610   -5.799507
frc3136   -5.341200
frc122    -4.895254
frc5804   -4.780722
frc5546   -4.345889
frc977    -4.056289
frc1522   -3.701569
frc5950   -3.289829
frc2890   -3.040691
frc2028   -3.007932
frc4638   -2.881242
frc6882   -2.218719
frc422    -2.159985
frc1793   -2.108951
frc1446   -2.041804
frc3072   -1.999357
frc6194   -1.924406
frc6312   -1.195891
frc5724   -1.014934
frc1137   -0.957798
frc5957   -0.936671
frc2068   -0.346741
frc6189   -0.330207
frc1908    0.165340
frc3389    0.237436
frc6021    0.578811
frc1080    0.658195
frc2998    1.018398
frc435     1.161463
frc4286    1.976426
frc539     2.516372
frc7330    2.854894
frc617     3.186812
frc5954    3.390377
frc3359    3.529944
dtype: float64

In [95]:
comp.sort_values('bayes')

,totalPoints,bayes
frc5954,4.719640,8.110017
frc7330,10.521972,13.376867
frc617,11.857770,15.044582
frc3359,15.220941,18.750886
frc539,28.118344,30.634716
frc4286,32.526832,34.503258
frc6021,40.503508,41.082319
frc435,43.106824,44.268287
frc6189,52.044510,51.714302
frc2998,55.581300,56.599698
